# Sphericity Validation

Here we perform several tests between DO/Successive Orders in Spherical/Plane Parallel modes in Nadir viewing geometry to ensure that they are consistent.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sasktran2 as sk


# Aerosol extinction profile
ext = np.array([5.79533614e-06, 8.92733941e-06, 1.35389309e-05, 2.02153233e-05,
       2.97190344e-05, 4.30214367e-05, 6.13326540e-05, 8.61285114e-05,
       1.19175875e-04, 1.62561620e-04, 2.18737907e-04, 2.90607016e-04,
       3.81680091e-04, 4.96356062e-04, 6.40368935e-04, 8.21442574e-04,
       1.05015110e-03, 1.34091527e-03, 1.71295710e-03, 2.19090632e-03,
       2.80462671e-03, 3.58776678e-03, 4.57458664e-03, 5.79483435e-03,
       7.26685790e-03, 8.98968754e-03, 1.09354337e-02, 1.30437771e-02,
       1.52204498e-02, 1.73411825e-02, 1.92617476e-02, 2.08333153e-02,
       2.19211122e-02, 2.24231510e-02, 2.22855062e-02, 2.15109810e-02,
       2.01593349e-02, 1.83389485e-02, 1.61916004e-02, 1.38734351e-02,
       1.15356743e-02, 9.30842011e-03, 7.28980709e-03, 5.54143568e-03,
       4.08962963e-03, 2.93106923e-03, 2.04088271e-03, 1.38128968e-03,
       9.09328956e-04, 5.82787062e-04, 3.64036678e-04, 2.21950686e-04,
       1.32323301e-04, 7.73148786e-05, 4.43928675e-05, 2.51275869e-05,
       1.40695661e-05, 7.82086681e-06, 4.33042737e-06, 2.39498866e-06,
       1.32541106e-06, 7.34412596e-07, 4.07222658e-07, 2.25617611e-07,
       1.24637971e-07, 6.84934738e-08, 3.73580758e-08, 2.01829176e-08,
       1.07826378e-08, 5.68908388e-09])

altitude = np.array([ 5. ,  5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ,
       10.5, 11. , 11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5,
       16. , 16.5, 17. , 17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5, 21. ,
       21.5, 22. , 22.5, 23. , 23.5, 24. , 24.5, 25. , 25.5, 26. , 26.5,
       27. , 27.5, 28. , 28.5, 29. , 29.5, 30. , 30.5, 31. , 31.5, 32. ,
       32.5, 33. , 33.5, 34. , 34.5, 35. , 35.5, 36. , 36.5, 37. , 37.5,
       38. , 38.5, 39. , 39.5]) * 1000

refractive = sk.mie.refractive.H2SO4()
dist = sk.mie.distribution.LogNormalDistribution().freeze(median_radius=200, mode_width=1.6)

aerosol_optical = sk.optical.Mie(dist, refractive)

# Test 1

In [ ]:
agrid = np.arange(0, 65000, 500.0)

config = sk.Config()
config.multiple_scatter_source = sk.MultipleScatterSource.DiscreteOrdinates
config.single_scatter_source = sk.SingleScatterSource.DiscreteOrdinates
config.num_sza = 1
config.num_singlescatter_moments = 64
config.num_forced_azimuth = 1


model_geometry = sk.Geometry1D(cos_sza=0.6,
                                solar_azimuth=0,
                                earth_radius_m=6372000,
                                altitude_grid_m=agrid,
                                interpolation_method=sk.InterpolationMethod.LinearInterpolation,
                                geometry_type=sk.GeometryType.PlaneParallel)


viewing_geo = sk.ViewingGeometry()

viewing_geo.add_ray(sk.GroundViewingSolar(0.6, 0, 1.0, 200000))

wavel = np.array([550])
atmosphere = sk.Atmosphere(model_geometry, config, wavelengths_nm=wavel)
atmosphere["surface"] = sk.constituent.LambertianSurface(0.3)

sk.climatology.us76.add_us76_standard_atmosphere(atmosphere)

atmosphere['rayleigh'] = sk.constituent.Rayleigh()
atmosphere["aerosol"] = sk.constituent.ExtinctionScatterer(aerosol_optical, altitude, ext / 1000, extinction_wavelength_nm=550)

engine = sk.Engine(config, model_geometry, viewing_geo)
output_do = engine.calculate_radiance(atmosphere)



config = sk.Config()

model_geometry = sk.Geometry1D(cos_sza=0.6,
                                solar_azimuth=0,
                                earth_radius_m=6372000,
                                altitude_grid_m=agrid,
                                interpolation_method=sk.InterpolationMethod.LinearInterpolation,
                                geometry_type=sk.GeometryType.PlaneParallel)

config.multiple_scatter_source = sk.MultipleScatterSource.SuccessiveOrders
config.single_scatter_source = sk.SingleScatterSource.Exact
config.num_singlescatter_moments = 64

engine = sk.Engine(config, model_geometry, viewing_geo)
output_hr = engine.calculate_radiance(atmosphere)


config = sk.Config()

model_geometry = sk.Geometry1D(cos_sza=0.6,
                                solar_azimuth=0,
                                earth_radius_m=6372000,
                                altitude_grid_m=agrid,
                                interpolation_method=sk.InterpolationMethod.LinearInterpolation,
                                geometry_type=sk.GeometryType.Spherical)

config.multiple_scatter_source = sk.MultipleScatterSource.DiscreteOrdinates
config.single_scatter_source = sk.SingleScatterSource.Exact
config.num_singlescatter_moments = 64

engine = sk.Engine(config, model_geometry, viewing_geo)
output_do_spher = engine.calculate_radiance(atmosphere)

config = sk.Config()

model_geometry = sk.Geometry1D(cos_sza=0.6,
                                solar_azimuth=0,
                                earth_radius_m=6372000,
                                altitude_grid_m=agrid,
                                interpolation_method=sk.InterpolationMethod.LinearInterpolation,
                                geometry_type=sk.GeometryType.Spherical)

config.multiple_scatter_source = sk.MultipleScatterSource.SuccessiveOrders
config.single_scatter_source = sk.SingleScatterSource.Exact
config.num_singlescatter_moments = 64

engine = sk.Engine(config, model_geometry, viewing_geo)
output_hr_spher = engine.calculate_radiance(atmosphere)

In [ ]:
output_do_spher["radiance"] / output_hr["radiance"]

In [ ]:
output_do["radiance"] / output_hr["radiance"]

In [ ]:
output_hr_spher["radiance"] / output_hr["radiance"]